#**Appendix B - Python Code**
The appendix shows the completed process of creating tables, inserting synthetic data, and conducting five analyses.

# Importing Libraries

In [ ]:
import sqlite3
import csv
import pandas as pd
from google.colab import files
from tabulate import tabulate

# Connect to the SQLite Database

In [ ]:
# Connect to the 'fun_rides_car_rental.db' SQLite database
conn = sqlite3.connect('fun_rides_car_rental.db')
# Create a cursor for executing SQL queries
cursor = conn.cursor()

# CREATE TABLE Queries

In [ ]:
# Create the "customers" table
cursor.execute('''
CREATE TABLE IF NOT EXISTS customers (
    customer_id TEXT PRIMARY KEY NOT NULL,
    first_name TEXT NOT NULL,
    last_name TEXT NOT NULL,
    sex TEXT NOT NULL CHECK (sex IN ('M', 'F')),
    birthdate DATE NOT NULL,
    driver_license_id TEXT NOT NULL,
    issue_country TEXT NOT NULL,
    country_code TEXT NOT NULL,
    phone TEXT NOT NULL,
    email TEXT NULL,
    street TEXT NOT NULL,
    city TEXT NOT NULL,
    county TEXT NULL,
    postcode TEXT NOT NULL,
    join_date DATE NOT NULL
);
''')

# Create the "rentals" table
cursor.execute('''
CREATE TABLE IF NOT EXISTS rentals (
    rental_id INTEGER PRIMARY KEY NOT NULL,
    customer_id TEXT NOT NULL,
    vehicle_id TEXT NOT NULL,
    rental_start_date DATE NOT NULL,
    rental_end_date DATE NOT NULL,
    rental_duration INTEGER NOT NULL,
    discount REAL NOT NULL DEFAULT 0,
    FOREIGN KEY (customer_id) REFERENCES customers(customer_id),
    FOREIGN KEY (vehicle_id) REFERENCES vehicles(vehicle_id)
);
''')

# Create the "payments" table
cursor.execute('''
CREATE TABLE IF NOT EXISTS payments (
    payment_id INTEGER PRIMARY KEY NOT NULL,
    rental_id INTEGER NOT NULL,
    payment_date DATE NOT NULL,
    payment_type TEXT NOT NULL CHECK (payment_type IN ('Credit Card', 'Debit Card', 'Cash', 'Bank Transfer')),
    payment_amount REAL NOT NULL,
    FOREIGN KEY (rental_id) REFERENCES rentals(rental_id)
);
''')

# Create the "vehicles" table
cursor.execute('''
CREATE TABLE IF NOT EXISTS vehicles (
    vehicle_id TEXT PRIMARY KEY NOT NULL,
    license_plate_number TEXT NOT NULL,
    vin TEXT NOT NULL,
    branch_id TEXT NOT NULL,
    make TEXT NOT NULL,
    model TEXT NOT NULL,
    manufacture_year INTEGER NOT NULL,
    current_mileage INTEGER NOT NULL,
    daily_rate REAL NOT NULL,
    deposit_amount REAL NOT NULL,
    vehicle_status TEXT NOT NULL CHECK (vehicle_status IN ('Available', 'Rented', 'Servicing', 'Totalled')),
    next_available_date DATE NULL,
    FOREIGN KEY (branch_id) REFERENCES branches(branch_id)
);
''')

# Create the "branches" table
cursor.execute('''
CREATE TABLE IF NOT EXISTS branches (
    branch_id TEXT PRIMARY KEY NOT NULL,
    branch_name TEXT NOT NULL,
    street TEXT NOT NULL,
    city TEXT NOT NULL,
    county TEXT NOT NULL,
    postcode TEXT NOT NULL,
    phone TEXT NOT NULL,
    email TEXT NOT NULL
);
''')

# Create the "insurance" table
cursor.execute('''
CREATE TABLE IF NOT EXISTS insurance (
    policy_number TEXT PRIMARY KEY NOT NULL,
    provider TEXT NOT NULL,
    vehicle_id TEXT NOT NULL,
    policy_type TEXT NOT NULL,
    premium_amount REAL NOT NULL,
    start_date DATE NOT NULL,
    end_date DATE NOT NULL,
    FOREIGN KEY (vehicle_id) REFERENCES vehicles(vehicle_id)
);
''')

# Create the "maintenance" table
cursor.execute('''
CREATE TABLE IF NOT EXISTS maintenance (
    maintenance_id TEXT PRIMARY KEY NOT NULL,
    vehicle_id TEXT NOT NULL,
    service_date DATE NOT NULL,
    service_end_date DATE NULL,
    service_duration INTEGER NULL,
    service_description TEXT NULL,
    service_cost REAL NULL,
    odometer_reading INTEGER NOT NULL,
    service_center_id TEXT NOT NULL,
    next_maintenance_due INTEGER NOT NULL,
    FOREIGN KEY (vehicle_id) REFERENCES vehicles(vehicle_id)
);
''')

# Create the "accidents" table
cursor.execute('''
CREATE TABLE IF NOT EXISTS accidents (
    accident_id TEXT PRIMARY KEY NOT NULL,
    vehicle_id TEXT NOT NULL,
    accident_date DATE NOT NULL,
    accident_description TEXT NOT NULL,
    repair_start_date DATE NULL,
    repair_end_date DATE NULL,
    repair_duration INTEGER NULL,
    repair_cost REAL NULL,
    insurance_claim_number TEXT NULL,
    claim_status TEXT NULL,
    FOREIGN KEY (vehicle_id) REFERENCES vehicles(vehicle_id)
);
''')

# Create the "additional_services" table
cursor.execute('''
CREATE TABLE IF NOT EXISTS additional_services (
    additional_service_id INTEGER PRIMARY KEY NOT NULL,
    additional_service_name TEXT NOT NULL,
    price REAL NULL,
    pricing_unit TEXT NULL
);
''')

# Create the "rental_additional_services" table (junction table)
cursor.execute('''
CREATE TABLE IF NOT EXISTS rental_additional_services (
    rental_id INTEGER NOT NULL,
    additional_service_id INTEGER NOT NULL,
    discount REAL NOT NULL DEFAULT 0,
    final_price REAL NOT NULL,
    PRIMARY KEY (rental_id, additional_service_id),
    FOREIGN KEY (rental_id) REFERENCES rentals(rental_id),
    FOREIGN KEY (additional_service_id) REFERENCES additional_services(additional_service_id)
);
''')

conn.commit()
print("Database and tables created successfully.")

Database and tables created successfully.


# Database Schema *Check*
(Outputs are Presented in Page 4 - 8)

In [ ]:
# Get table names from the schema
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
tables = cursor.fetchall()

for table_name in tables:
    table = table_name[0]
    print(f"Table: {table}")

    # Get column info for the current table
    cursor.execute(f"PRAGMA table_info({table});")
    columns = cursor.fetchall()

    # Get foreign key info for the current table
    cursor.execute(f"PRAGMA foreign_key_list({table});")
    foreign_keys = cursor.fetchall()

    # foreign_keys tuple structure: (id, seq, ref_table, fk_from, ref_column, on_update, on_delete, match)
    # Build a dictionary mapping each foreign key column to its referenced table(s)
    fk_ref = {}
    for fk in foreign_keys:
        col_from = fk[3]      # the "from" column in the current table
        ref_table = fk[2]     # the referenced table name
        if col_from in fk_ref:
            fk_ref[col_from].append(ref_table)
        else:
            fk_ref[col_from] = [ref_table]

    # Define headers for tabulate output
    headers = ["Column", "Type", "Not Null", "Default Value", "Primary Key", "Foreign Key", "Reference Table"]
    rows = []
    for col in columns:
        # col structure: (cid, name, type, notnull, dflt_value, pk)
        column_name = col[1]
        is_fk = 1 if column_name in fk_ref else 0
        ref_tables = ", ".join(fk_ref[column_name]) if column_name in fk_ref else ""
        rows.append([column_name, col[2], col[3], col[4], col[5], is_fk, ref_tables])

    # Print the table information using tabulate
    print(tabulate(rows, headers=headers, tablefmt="pretty"))
    print("-" * 40)

Table: customers
+-------------------+------+----------+---------------+-------------+-------------+-----------------+
|      Column       | Type | Not Null | Default Value | Primary Key | Foreign Key | Reference Table |
+-------------------+------+----------+---------------+-------------+-------------+-----------------+
|    customer_id    | TEXT |    1     |               |      1      |      0      |                 |
|    first_name     | TEXT |    1     |               |      0      |      0      |                 |
|     last_name     | TEXT |    1     |               |      0      |      0      |                 |
|        sex        | TEXT |    1     |               |      0      |      0      |                 |
|     birthdate     | DATE |    1     |               |      0      |      0      |                 |
| driver_license_id | TEXT |    1     |               |      0      |      0      |                 |
|   issue_country   | TEXT |    1     |               |      0   

# Load the Synthetic Data

In [ ]:
# Upload the synthetic data
uploaded = files.upload()

Saving accidents.csv to accidents.csv
Saving additional_services.csv to additional_services.csv
Saving branches.csv to branches.csv
Saving customers.csv to customers.csv
Saving insurance.csv to insurance.csv
Saving maintenance.csv to maintenance.csv
Saving payments.csv to payments.csv
Saving rental_additional_services.csv to rental_additional_services.csv
Saving rentals.csv to rentals.csv
Saving vehicles.csv to vehicles.csv


In [ ]:
def import_csv_to_table(csv_file, table_name):
    # Open the CSV file and create a reader, skipping the header row
    with open(csv_file, 'r', encoding='utf-8') as file:
        csv_reader = csv.reader(file)
        next(csv_reader)  # Skip header row if present
        # For each row in the CSV, build and execute an INSERT query
        for row in csv_reader:
            placeholders = ', '.join(['?' for _ in row])
            sql = f"INSERT INTO {table_name} VALUES ({placeholders})"
            cursor.execute(sql, row)

try:
    import_csv_to_table('branches.csv', 'branches')
    import_csv_to_table('vehicles.csv', 'vehicles')
    import_csv_to_table('customers.csv', 'customers')
    import_csv_to_table('additional_services.csv', 'additional_services')
    import_csv_to_table('rentals.csv', 'rentals')
    import_csv_to_table('payments.csv', 'payments')
    import_csv_to_table('accidents.csv', 'accidents')
    import_csv_to_table('maintenance.csv', 'maintenance')
    import_csv_to_table('insurance.csv', 'insurance')
    import_csv_to_table('rental_additional_services.csv', 'rental_additional_services')

    conn.commit()
    print("Data imported successfully!")
except Exception as e:
    print(f"An error occurred: {e}")
    conn.rollback()

Data imported successfully!


# Data Integrity Checks

## Record Count Check

In [ ]:
csv_files = {
    "customers": "customers.csv",
    "rentals": "rentals.csv",
    "payments": "payments.csv",
    "vehicles": "vehicles.csv",
    "branches": "branches.csv",
    "insurance": "insurance.csv",
    "maintenance": "maintenance.csv",
    "accidents": "accidents.csv",
    "additional_services": "additional_services.csv",
    "rental_additional_services": "rental_additional_services.csv"
}


# Dictionary to store record counts
csv_counts = {}

# Count the number of records in each CSV file
for table, file_path in csv_files.items():
    try:
        df = pd.read_csv(file_path)
        csv_counts[table] = len(df)
    except Exception as e:
        csv_counts[table] = f"Error: {e}"


# List of tables to check for record counts
tables_to_check = [
    'customers',
    'rentals',
    'payments',
    'vehicles',
    'branches',
    'insurance',
    'maintenance',
    'accidents',
    'additional_services',
    'rental_additional_services'
]

# Dictionary to store database record counts
db_counts = {}


# Iterate through each table in the list
for table in tables_to_check:
    try:
        # Execute a SQL query to count the records in the current table
        df_count = pd.read_sql_query(f"SELECT COUNT(*) AS total_records FROM {table};", conn)
        db_counts[table] = df_count['total_records'].iloc[0]
    except Exception as e:
        db_counts[table] = f"Error: {e}"

# Create a DataFrame for comparison
comparison_df = pd.DataFrame({
    "Table": tables_to_check,
    "CSV Record Count": [csv_counts.get(table, "N/A") for table in tables_to_check],
    "DB Record Count": [db_counts.get(table, "N/A") for table in tables_to_check]
})

print(comparison_df)

                        Table  CSV Record Count  DB Record Count
0                   customers             22498            22498
1                     rentals             27539            27539
2                    payments             33182            33182
3                    vehicles               150              150
4                    branches                10               10
5                   insurance               587              587
6                 maintenance               726              726
7                   accidents                32               32
8         additional_services                 7                7
9  rental_additional_services             80068            80068


## Unique Primary Key Check

In [ ]:
# Map each table to its primary key column(s). For composite keys, columns are concatenated
tables_and_keys = {
    'accidents': 'accident_id',
    'additional_services': 'additional_service_id',
    'branches': 'branch_id',
    'customers': 'customer_id',
    'insurance': 'policy_number',
    'maintenance': 'maintenance_id',
    'payments': 'payment_id',
    'rental_additional_services': 'rental_id || additional_service_id',  # Composite key
    'rentals': 'rental_id',
    'vehicles': 'vehicle_id'
}

# Loop through each table, grouping by the primary key and checking for duplicates
for table, key in tables_and_keys.items():
    query = f"""
    SELECT {key} AS key_val, COUNT(*) AS count
    FROM {table}
    GROUP BY key_val
    HAVING COUNT(*) > 1;
    """

    # Execute the query and fetch results
    cursor.execute(query)
    duplicates = cursor.fetchall()

    # Output the results
    if duplicates:
        print(f"Duplicates found in table '{table}':")
        for row in duplicates:
            print(row)
    else:
        print(f"No duplicates in table '{table}'.")

No duplicates in table 'accidents'.
No duplicates in table 'additional_services'.
No duplicates in table 'branches'.
No duplicates in table 'customers'.
No duplicates in table 'insurance'.
No duplicates in table 'maintenance'.
No duplicates in table 'payments'.
No duplicates in table 'rental_additional_services'.
No duplicates in table 'rentals'.
No duplicates in table 'vehicles'.


## Invalid Value Check

In [ ]:
# Check for invalid sex values in customers
cursor.execute("SELECT customer_id, sex FROM customers WHERE sex NOT IN ('M', 'F');")
invalid_sex = cursor.fetchall()
if invalid_sex:
    print("Invalid sex values found in customers:")
    for row in invalid_sex:
        print(row)
else:
    print("All 'sex' values in customers are valid.")

# Check for invalid vehicle_status in vehicles
cursor.execute("""
    SELECT vehicle_id, vehicle_status
    FROM vehicles
    WHERE vehicle_status NOT IN ('Available', 'Rented', 'Servicing', 'Totalled');
""")
invalid_status = cursor.fetchall()
if invalid_status:
    print("Invalid vehicle_status values found in vehicles:")
    for row in invalid_status:
        print(row)
else:
    print("All 'vehicle_status' values in vehicles are valid.")

# Check for invalid payment_type in payments
cursor.execute("""
    SELECT payment_id, payment_type
    FROM payments
    WHERE payment_type NOT IN ('Credit Card', 'Debit Card', 'Bank Transfer', 'Cash');
""")
invalid_payment = cursor.fetchall()
if invalid_payment:
    print("Invalid payment_type values found in payments:")
    for row in invalid_payment:
        print(row)
else:
    print("All 'payment_type' values in payments are valid.")

All 'sex' values in customers are valid.
All 'vehicle_status' values in vehicles are valid.
All 'payment_type' values in payments are valid.


## Rentals Integrity Check

SQL query to find overlapping rentals for the same vehicle using date comparisons

In [ ]:
query = """
SELECT
  r1.rental_id AS rental1,
  r2.rental_id AS rental2,
  r1.vehicle_id,
  date(r1.rental_start_date) AS start1, date(r1.rental_end_date) AS end1,
  date(r2.rental_start_date) AS start2, date(r2.rental_end_date) AS end2
FROM rentals r1
JOIN rentals r2
  ON r1.vehicle_id = r2.vehicle_id
 AND r1.rental_id < r2.rental_id
WHERE date(r1.rental_start_date) <= date(r2.rental_end_date)
  AND date(r2.rental_start_date) <= date(r1.rental_end_date);
"""

# Execute the query and fetch results
cursor.execute(query)
overlaps = cursor.fetchall()

# Output the results
if overlaps:
    print("Overlapping rentals for the same vehicle:")
    for overlap in overlaps:
        print(overlap)
else:
    print("No overlapping rentals for the same vehicle found.")


No overlapping rentals for the same vehicle found.


SQL query to find overlapping rentals for the same customer using date comparisons

In [ ]:
query = """
SELECT
  r1.rental_id AS rental1,
  r2.rental_id AS rental2,
  r1.customer_id,
  date(r1.rental_start_date) AS start1, date(r1.rental_end_date) AS end1,
  date(r2.rental_start_date) AS start2, date(r2.rental_end_date) AS end2
FROM rentals r1
JOIN rentals r2
  ON r1.customer_id = r2.customer_id
 AND r1.rental_id < r2.rental_id
WHERE date(r1.rental_start_date) <= date(r2.rental_end_date)
  AND date(r2.rental_start_date) <= date(r1.rental_end_date);
"""

# Execute the query and fetch results
cursor.execute(query)
overlapping_customer_rentals = cursor.fetchall()

# Output the results
if overlapping_customer_rentals:
    print("Overlapping rentals for the same customer:")
    for overlap in overlapping_customer_rentals:
        print(overlap)
else:
    print("No overlapping rentals for the same customer found.")

No overlapping rentals for the same customer found.


# Business Reports
These queries have output several tables and these tables are presented in the Business Report section.

## Financial Analysis
### 1. Branches Revenue Report 2024

In [ ]:
# Define the query
query = """
SELECT
    b.branch_id,
    b.branch_name,
    SUM(CASE WHEN strftime('%m',
         substr(p.payment_date, 7, 4) || '-' || substr(p.payment_date, 4, 2) || '-' || substr(p.payment_date, 1, 2)
         ) = '01' THEN p.payment_amount ELSE 0 END) AS Jan_2024,
    SUM(CASE WHEN strftime('%m',
         substr(p.payment_date, 7, 4) || '-' || substr(p.payment_date, 4, 2) || '-' || substr(p.payment_date, 1, 2)
         ) = '02' THEN p.payment_amount ELSE 0 END) AS Feb_2024,
    SUM(CASE WHEN strftime('%m',
         substr(p.payment_date, 7, 4) || '-' || substr(p.payment_date, 4, 2) || '-' || substr(p.payment_date, 1, 2)
         ) = '03' THEN p.payment_amount ELSE 0 END) AS Mar_2024,
    SUM(CASE WHEN strftime('%m',
         substr(p.payment_date, 7, 4) || '-' || substr(p.payment_date, 4, 2) || '-' || substr(p.payment_date, 1, 2)
         ) = '04' THEN p.payment_amount ELSE 0 END) AS Apr_2024,
    SUM(CASE WHEN strftime('%m',
         substr(p.payment_date, 7, 4) || '-' || substr(p.payment_date, 4, 2) || '-' || substr(p.payment_date, 1, 2)
         ) = '05' THEN p.payment_amount ELSE 0 END) AS May_2024,
    SUM(CASE WHEN strftime('%m',
         substr(p.payment_date, 7, 4) || '-' || substr(p.payment_date, 4, 2) || '-' || substr(p.payment_date, 1, 2)
         ) = '06' THEN p.payment_amount ELSE 0 END) AS Jun_2024,
    SUM(CASE WHEN strftime('%m',
         substr(p.payment_date, 7, 4) || '-' || substr(p.payment_date, 4, 2) || '-' || substr(p.payment_date, 1, 2)
         ) = '07' THEN p.payment_amount ELSE 0 END) AS Jul_2024,
    SUM(CASE WHEN strftime('%m',
         substr(p.payment_date, 7, 4) || '-' || substr(p.payment_date, 4, 2) || '-' || substr(p.payment_date, 1, 2)
         ) = '08' THEN p.payment_amount ELSE 0 END) AS Aug_2024,
    SUM(CASE WHEN strftime('%m',
         substr(p.payment_date, 7, 4) || '-' || substr(p.payment_date, 4, 2) || '-' || substr(p.payment_date, 1, 2)
         ) = '09' THEN p.payment_amount ELSE 0 END) AS Sep_2024,
    SUM(CASE WHEN strftime('%m',
         substr(p.payment_date, 7, 4) || '-' || substr(p.payment_date, 4, 2) || '-' || substr(p.payment_date, 1, 2)
         ) = '10' THEN p.payment_amount ELSE 0 END) AS Oct_2024,
    SUM(CASE WHEN strftime('%m',
         substr(p.payment_date, 7, 4) || '-' || substr(p.payment_date, 4, 2) || '-' || substr(p.payment_date, 1, 2)
         ) = '11' THEN p.payment_amount ELSE 0 END) AS Nov_2024,
    SUM(CASE WHEN strftime('%m',
         substr(p.payment_date, 7, 4) || '-' || substr(p.payment_date, 4, 2) || '-' || substr(p.payment_date, 1, 2)
         ) = '12' THEN p.payment_amount ELSE 0 END) AS Dec_2024,
    SUM(p.payment_amount) AS Total_Annual_Revenue
FROM branches b
JOIN vehicles v ON b.branch_id = v.branch_id
JOIN rentals r ON v.vehicle_id = r.vehicle_id
JOIN payments p ON r.rental_id = p.rental_id
WHERE (substr(p.payment_date, 7, 4) || '-' || substr(p.payment_date, 4, 2) || '-' || substr(p.payment_date, 1, 2))
      BETWEEN '2024-01-01' AND '2024-12-31'
GROUP BY
    b.branch_id,
    b.branch_name
ORDER BY
    b.branch_id;
"""

# Execute the query and load the results into a DataFrame
df_revenue = pd.read_sql_query(query, conn)

# Print the report
print("Annual Revenue Report for 2024 by Branch:")
print(df_revenue)

Annual Revenue Report for 2024 by Branch:
  branch_id              branch_name  Jan_2024  Feb_2024  Mar_2024  Apr_2024  \
0    FR0001  FunRides London Central  49321.52  38968.74  47968.45  44645.63   
1    FR0002     FunRides London West  34139.18  29677.20  30181.41  31317.01   
2    FR0003     FunRides London East  30037.63  23066.92  28509.92  24330.85   
3    FR0004      FunRides Manchester  40113.01  45927.25  42724.32  46055.90   
4    FR0005      FunRides Birmingham  44012.80  38446.67  32089.35  47372.24   
5    FR0006         FunRides Glasgow   6902.51  14075.50  13230.86  22661.21   
6    FR0007       FunRides Edinburgh      0.00   1935.31   1948.13   1416.44   
7    FR0008         FunRides Bristol      0.00      0.00      0.00      0.00   

   May_2024  Jun_2024  Jul_2024  Aug_2024  Sep_2024  Oct_2024  Nov_2024  \
0  40774.26  39679.46  57774.87  57366.90  52503.60  60929.84  50413.22   
1  32959.23  28044.82  42962.38  39185.80  40697.38  34952.56  40808.45   
2  22610.18 

### 2. Year-Over-Year (YoY) and Quarter-over-Quarter (QoQ) Revenue Growth Report

In [ ]:
# Define the query
query = """
WITH quarterly_revenue AS (
    SELECT
        CAST(substr(r.rental_start_date, 7, 4) AS INTEGER) AS rental_year,
        CASE
            WHEN CAST(substr(r.rental_start_date, 4, 2) AS INTEGER) BETWEEN 1 AND 3 THEN 'Q1'
            WHEN CAST(substr(r.rental_start_date, 4, 2) AS INTEGER) BETWEEN 4 AND 6 THEN 'Q2'
            WHEN CAST(substr(r.rental_start_date, 4, 2) AS INTEGER) BETWEEN 7 AND 9 THEN 'Q3'
            ELSE 'Q4'
        END AS rental_quarter,
        SUM(p.payment_amount) AS total_revenue
    FROM rentals r
    JOIN payments p ON r.rental_id = p.rental_id
    JOIN rental_additional_services ras ON r.rental_id = ras.rental_id
    JOIN additional_services as2 ON ras.additional_service_id = as2.additional_service_id
    GROUP BY rental_year, rental_quarter
)

SELECT
    curr.rental_year || '-' || curr.rental_quarter AS rental_period,
    curr.total_revenue,

    -- YoY Calculation
    prev_year.total_revenue AS prev_year_revenue,
    CASE
        WHEN prev_year.total_revenue IS NOT NULL
        THEN ROUND(((curr.total_revenue - prev_year.total_revenue) / prev_year.total_revenue) * 100, 2)
        ELSE NULL
    END AS yoy_growth_percentage,

    -- QoQ Calculation
    prev_quarter.total_revenue AS prev_quarter_revenue,
    CASE
        WHEN prev_quarter.total_revenue IS NOT NULL
        THEN ROUND(((curr.total_revenue - prev_quarter.total_revenue) / prev_quarter.total_revenue) * 100, 2)
        ELSE NULL
    END AS qoq_growth_percentage

FROM quarterly_revenue curr

-- Join for YoY growth (previous year, same quarter)
LEFT JOIN quarterly_revenue prev_year
ON curr.rental_quarter = prev_year.rental_quarter
AND curr.rental_year = prev_year.rental_year + 1

-- Join for QoQ growth (previous quarter, same year or previous year if crossing Q4 to Q1)
LEFT JOIN quarterly_revenue prev_quarter
ON (
    (curr.rental_year = prev_quarter.rental_year AND
        ((curr.rental_quarter = 'Q2' AND prev_quarter.rental_quarter = 'Q1') OR
         (curr.rental_quarter = 'Q3' AND prev_quarter.rental_quarter = 'Q2') OR
         (curr.rental_quarter = 'Q4' AND prev_quarter.rental_quarter = 'Q3')))
    OR
    (curr.rental_quarter = 'Q1' AND prev_quarter.rental_quarter = 'Q4' AND curr.rental_year = prev_quarter.rental_year + 1)
)

ORDER BY curr.rental_year, curr.rental_quarter;
"""

# Execute the query and load the results into a DataFrame
df_annual_rev = pd.read_sql_query(query, conn)

# Print the report
print("Annual Revenue Report:")
print(df_annual_rev)

Annual Revenue Report:
   rental_period  total_revenue  prev_year_revenue  yoy_growth_percentage  \
0        2020-Q3      460090.31                NaN                    NaN   
1        2020-Q4      358941.88                NaN                    NaN   
2        2021-Q1      409059.36                NaN                    NaN   
3        2021-Q2      515738.19                NaN                    NaN   
4        2021-Q3      916218.27          460090.31                  99.14   
5        2021-Q4      874422.08          358941.88                 143.61   
6        2022-Q1      833120.83          409059.36                 103.67   
7        2022-Q2      946740.69          515738.19                  83.57   
8        2022-Q3     1485589.69          916218.27                  62.14   
9        2022-Q4     1285245.18          874422.08                  46.98   
10       2023-Q1     1331929.81          833120.83                  59.87   
11       2023-Q2     1432318.63          946740.69   

## Vehicle Utilisation Report

In [ ]:
# Define the query
query = """
SELECT
    b.branch_id,
    b.branch_name,
    AVG(
      CASE
         WHEN (366 - COALESCE(m.total_maint, 0) - COALESCE(a.total_acc, 0)) > 0
         THEN COALESCE(r.total_rented, 0) * 100.0 / (366 - COALESCE(m.total_maint, 0) - COALESCE(a.total_acc, 0))
         ELSE 0
      END
    ) AS vehicle_utilization
FROM vehicles v
JOIN branches b ON v.branch_id = b.branch_id
LEFT JOIN (
  SELECT
    vehicle_id,
    SUM(
      CASE
         WHEN (substr(rental_end_date, 7, 4) || '-' || substr(rental_end_date, 4, 2) || '-' || substr(rental_end_date, 1, 2)) > '2024-12-31'
         THEN julianday('2024-12-31') - julianday(substr(rental_start_date, 7, 4) || '-' || substr(rental_start_date, 4, 2) || '-' || substr(rental_start_date, 1, 2)) + 1
         ELSE rental_duration
      END
    ) AS total_rented
  FROM rentals
  WHERE (substr(rental_start_date, 7, 4) || '-' || substr(rental_start_date, 4, 2) || '-' || substr(rental_start_date, 1, 2))
         BETWEEN '2024-01-01' AND '2024-12-31'
  GROUP BY vehicle_id
) r ON v.vehicle_id = r.vehicle_id
LEFT JOIN (
  SELECT vehicle_id, SUM(service_duration) AS total_maint
  FROM maintenance
  WHERE (substr(service_date, 7, 4) || '-' || substr(service_date, 4, 2) || '-' || substr(service_date, 1, 2))
         BETWEEN '2024-01-01' AND '2024-12-31'
  GROUP BY vehicle_id
) m ON v.vehicle_id = m.vehicle_id
LEFT JOIN (
  SELECT vehicle_id, SUM(repair_duration) AS total_acc
  FROM accidents
  WHERE (substr(accident_date, 7, 4) || '-' || substr(accident_date, 4, 2) || '-' || substr(accident_date, 1, 2))
         BETWEEN '2024-01-01' AND '2024-12-31'
  GROUP BY vehicle_id
) a ON v.vehicle_id = a.vehicle_id
GROUP BY b.branch_id, b.branch_name
ORDER BY b.branch_id;
"""

# Execute the query and load the results into a DataFrame
df_utilization = pd.read_sql_query(query, conn)

# Print the report
print("Vehicle Utilization Report for 2024):")
print(df_utilization)

Vehicle Utilization Report for 2024):
  branch_id              branch_name  vehicle_utilization
0    FR0001  FunRides London Central            83.641111
1    FR0002     FunRides London West            92.241893
2    FR0003     FunRides London East            92.344693
3    FR0004      FunRides Manchester            78.362525
4    FR0005      FunRides Birmingham            72.448054
5    FR0006         FunRides Glasgow            42.697652
6    FR0007       FunRides Edinburgh            14.037458
7    FR0008         FunRides Bristol             1.402613
8    FR0009       FunRides Liverpool             0.000000
9    FR0010       FunRides Newcastle             0.000000


## Demand Analysis

### 1. Customer Demographics

In [ ]:
# Define the query
query = """
SELECT
    CASE
        WHEN (2024 - CAST(substr(birthdate, 7, 4) AS INTEGER)) BETWEEN 18 AND 24 THEN '18-24'
        WHEN (2024 - CAST(substr(birthdate, 7, 4) AS INTEGER)) BETWEEN 25 AND 34 THEN '25-34'
        WHEN (2024 - CAST(substr(birthdate, 7, 4) AS INTEGER)) BETWEEN 35 AND 44 THEN '35-44'
        WHEN (2024 - CAST(substr(birthdate, 7, 4) AS INTEGER)) BETWEEN 45 AND 54 THEN '45-54'
        ELSE '55+'
    END AS age_group,
    COUNT(*) AS total_customers
FROM customers
GROUP BY age_group
ORDER BY age_group;
"""

# Execute the query and load the results into a DataFrame
df_age_group = pd.read_sql_query(query, conn)

#Print the report
print("Rental counts by age group for each vehicle:")
print(df_age_group)

Rental counts by age group for each vehicle:
  age_group  total_customers
0     18-24             3008
1     25-34             5228
2     35-44             4831
3     45-54             4802
4       55+             4629


### 2. Vehicle Demand by Gender

In [ ]:
# Define the query
query = """
WITH rental_count AS (
    SELECT
        r.vehicle_id,
        c.sex,
        COUNT(r.rental_id) AS total_rentals
    FROM rentals r
    JOIN customers c ON r.customer_id = c.customer_id
    GROUP BY r.vehicle_id, c.sex
)
SELECT
    v.make,
    v.model,
    rc.sex,
    rc.total_rentals
FROM rental_count rc
JOIN vehicles v ON rc.vehicle_id = v.vehicle_id
GROUP BY v.make, v.model, rc.sex
ORDER BY v.make, v.model, rc.sex;
"""

# Execute the query and load the results into a DataFrame
df_rental_by_sex = pd.read_sql_query(query, conn)

# Print the report
print("Rental Counts by Vehicle and Customer Sex:")
print(df_rental_by_sex)

Rental Counts by Vehicle and Customer Sex:
          make     model sex  total_rentals
0         Audi        A3   F             17
1         Audi        A3   M             23
2         Audi        A4   F            213
3         Audi        A4   M            230
4         Audi        Q5   F            108
5         Audi        Q5   M            130
6          BMW  3 Series   F             16
7          BMW  3 Series   M             18
8          BMW  5 Series   F             46
9          BMW  5 Series   M             39
10         BMW        X3   F            185
11         BMW        X3   M            166
12        Ford    Fiesta   F            177
13        Ford    Fiesta   M            180
14        Ford     Focus   F             46
15        Ford     Focus   M             44
16        Ford   Mustang   F            219
17        Ford   Mustang   M            261
18       Honda    Accord   F              1
19       Honda    Accord   M              2
20       Honda     Civic   F     

### 3. Average Rental Frequency

In [ ]:
# Define the query
query = """
WITH rental_count AS (
    SELECT
        vehicle_id,
        COUNT(*) AS rental_times
    FROM rentals
    WHERE rental_start_date >= '2024-01-01' AND rental_start_date <= '2024-12-31'
    GROUP BY vehicle_id
)
SELECT
    v.make,
    v.model,
	SUM(r.rental_times) AS total_rentals,
    COUNT(r.vehicle_id) AS total_vehicles,
    AVG(r.rental_times) AS avg_rental_frequency
FROM rental_count r
JOIN vehicles v ON r.vehicle_id = v.vehicle_id
GROUP BY v.make, v.model
ORDER BY avg_rental_frequency DESC;
"""

# Execute the query and load the results into a DataFrame
df_avg_rental = pd.read_sql_query(query, conn)

# Print the report
print("Average Rental Frequency Report:")
print(df_avg_rental)

Average Rental Frequency Report:
Empty DataFrame
Columns: [make, model, total_rentals, total_vehicles, avg_rental_frequency]
Index: []


In [ ]:
# Close the connection
conn.close()